In [1]:
%%javascript
utils.load_extension('collapsible_headings/main')
utils.load_extension('hide_input/main')
utils.load_extension('autosavetime/main')
utils.load_extension('execute_time/ExecuteTime')
utils.load_extension('code_prettify/code_prettify')
utils.load_extension('scroll_down/main')
utils.load_extension('jupyter-js-widgets/extension')

<IPython.core.display.Javascript object>

In [2]:
from IPython.display import display, HTML, clear_output
display(HTML("<style>.container { width:100% !important; }</style>"))

Ignacio Oguiza - email: oguiza@gmail.com

## Import libraries

In [3]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [4]:
from fastai_extensions import *

## Prepare data

In [5]:
train_size = 500
seed = 1234
bs = 128
path = untar_data(URLs.CIFAR)
tfms = get_transforms()

### Prepare simulated labeled/ unlabeled

In [6]:
y_train = ImageList.from_folder(path / 'train').split_none().label_from_folder().y.items # 50k
y_valid = ImageList.from_folder(path / 'test').split_none().label_from_folder().y.items  # 10k
l_idx, u_idx = random_strat_splitter(y_train, train_size=train_size, seed=seed)  # train_size, 50k - train_size
valid_idx, _ = random_strat_splitter(y_valid, train_size=train_size, seed=seed)  # 10k

### Prepare labeled databunch and list of unlabeled samples

In [7]:
data = (ItemLists('.',
                  ImageList.from_folder(path / 'train')[l_idx],  # labeled train (in this case a subset of the original)
                  ImageList.from_folder(path / 'test'))          # labeled valid
        .label_from_folder()
        .transform(tfms)
        .databunch(bs=bs, val_bs=bs * 2)
        .normalize(cifar_stats))
print(data, '\n')

ImageDataBunch;

Train: LabelList (500 items)
x: ImageList
Image (3, 32, 32),Image (3, 32, 32),Image (3, 32, 32),Image (3, 32, 32),Image (3, 32, 32)
y: CategoryList
deer,bird,truck,cat,deer
Path: /home/oguizadl/.fastai/data/cifar10/train;

Valid: LabelList (10000 items)
x: ImageList
Image (3, 32, 32),Image (3, 32, 32),Image (3, 32, 32),Image (3, 32, 32),Image (3, 32, 32)
y: CategoryList
ship,ship,ship,ship,ship
Path: /home/oguizadl/.fastai/data/cifar10/test;

Test: None 



In [8]:
# unlabeled list
ulist = ImageList.from_folder(path / 'train')[u_idx]
print(ulist, '\n')

ImageList (49500 items)
Image (3, 32, 32),Image (3, 32, 32),Image (3, 32, 32),Image (3, 32, 32),Image (3, 32, 32)
Path: /home/oguizadl/.fastai/data/cifar10/train 



## Build learner

In [9]:
model = models.WideResNet(num_groups=3, N=4, num_classes=10, k=2, start_nf=32)
learn = Learner(data, model, metrics=accuracy).mixmatch(ulist, α=.75, λ=75, num_workers=None)

## Train

In [ ]:
learn.fit_one_cycle(100)

labeled dataset     :           500 samples
unlabeled dataset   :        49,500 samples
total train samples :    14,822,400 samples


epoch,train_loss,valid_loss,accuracy,train_Lx,train_Lu*λ,time
0,2.774208,2.451387,0.174500,2.051836,0.722369,03:04
